In [6]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'

In [ ]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

In [3]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [4]:
df.head(10)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь
3,"«Сачхере» — «Абердин» И тот, и другой клуб оде...",1.45,Айнтрахт - 3 и более гола,Лига Ставок,2019-07-25 17:00:00,Лига Европы УЕФА,Флора,Айнтрахт Франкфурт
4,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
5,Июль подходит к концу. Скоро мадридское дерби ...,1.62,Атлетико забьет 1 или 2 гола: да,Фонбет,2019-07-27 00:06:00,Международный Кубок чемпионов,Реал Мадрид,Атлетико Мадрид
6,"«Ахмат», который доставил много проблем команд...",1.71,Ахмат забьет 1 или 2 гола: да,Пари-Матч,2019-07-14 18:30:00,Премьер-Лига,Ахмат,Краснодар
7,"Никто толком не представляет, в каких составах...",2.45,Бавария забьет в обоих таймах,Лига Ставок,2019-07-18 03:00:00,Международный Кубок чемпионов,Арсенал,Бавария
8,"Матч вроде бы товарищеский, но - это и репутац...",3.25,Барселона забьет в каждом тайме,Лига Ставок,2019-07-23 10:30:00,Клубные товарищеские матчи,Барселона,Челси
9,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония


In [7]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'
    
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))
    
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)

russian.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь


In [8]:
unique_leagues = russian['league_name_in_russian'].unique().tolist()
len(unique_leagues)

288

In [9]:
unique_teams1 = russian['team_of_season_1_name_in_russian'].unique().tolist()
len(unique_teams1)

3773

In [10]:
unique_teams2 = russian['team_of_season_2_name_in_russian'].unique().tolist()
len(unique_teams2)

3730

In [11]:
unique_teams = unique_teams1 + unique_teams2
len(set(unique_teams))

4422

In [12]:
russian['result.tsv'][5133]

'Швейцария - Англия Совершенно бестолковый матч. Команды без особой мотивации должны мучиться на жаре днем. Ну раз уж приехали в Португалию, чего бы не сыграть, по сути, товарищеский матч. Думаю, у Швейцарии преимущество должно быть. Англия играла на день позже, ей пришлось еще и овертайм провести. Плюс результат ударил по амбициям команды Саутгейта. Она упустила победу в основное время. Ее гол отменили из-за микроскопического офсайда. Ну и сами игроки наошибались от души. Швейцария проиграла тоже обидно. Но было больше времени, чтобы все это переварить. Ну и день отдыха лишний. К тому же, как мне кажется, для Швейцарии престижнее занять третье место в первой Лиге наций. Британцы же максималисты. Да и по качеству футбола Швейцария понравилась больше. Ей бы еще решить вопрос с мотивацией. Думаю, пока британцы будут разбираться со своим настроем, под шумок швейцарцы могут забить. Тем более в обороне англичан наверняка будет много перестановок. Нет смысла нагружать основных ребят, конец с

### Команда 1 и 2 (список + прямой мэтч)

In [20]:
teams1_match = np.zeros(df.shape[0])
teams2_match = np.zeros(df.shape[0])

https://stackoverflow.com/questions/15852436/how-to-find-the-2nd-max-of-a-counter-python

In [22]:
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            #print(token)
            if token in unique_teams:
                count_tokens.append(token)
        #print(count_tokens)
        c = Counter(count_tokens)
        #if count_tokens == 2
        #print(df['team_of_season_1_name_in_russian'][i], df['team_of_season_2_name_in_russian'][i])
        #print(c)
        #print(max(c))
        #print(c.most_common()[0:2])
        #print(text)
        try:
            max_w1 = sorted(c, key=c.get)[-1]
            #print(max_w1)
        except:
            continue
        try:
            max_w2 = sorted(c, key=c.get)[-2]
            #print(max_w2)
        except:
            continue
        
        #print('мэтч команды 1')
        if df['team_of_season_1_name_in_russian'][i] == max_w1 or df['team_of_season_1_name_in_russian'][i] == max_w2:
            teams1_match[i] = 1
            #print(df['team_of_season_1_name_in_russian'][i], max_w1)
            #print(df['team_of_season_1_name_in_russian'][i], max_w2)
        
        #print('мэтч команды 2')
        if df['team_of_season_2_name_in_russian'][i] == max_w1 or df['team_of_season_2_name_in_russian'][i] == max_w2:
            teams2_match[i] = 1
            #print(df['team_of_season_2_name_in_russian'][i], max_w1)
            #print(df['team_of_season_2_name_in_russian'][i], max_w2)
        
        

702908it [5:24:59, 36.05it/s] 


In [25]:
df['list_direct_match_for_teams1'] = teams1_match

In [26]:
df['list_direct_match_for_teams2'] = teams2_match

In [27]:
success1 = df[df['list_direct_match_for_teams1'] == 1].shape[0]
success2 = df[df['list_direct_match_for_teams2'] == 1].shape[0]

In [29]:
all_vals = df['list_direct_match_for_teams1'].shape[0]

In [30]:
result_team1 = success1 / all_vals
print(result_team1)

0.20481058687623416


In [31]:
result_team2 = success2 / all_vals
print(result_team2)

0.2051363763109824


### Лиги

In [33]:
leagues_match = np.zeros(df.shape[0])

In [ ]:
%%time 
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    n_words_league = len(word_tokenize(df['league_name_in_russian'][i]))
    N = n_words_league
    words = word_tokenize(text)
    words = [word.lower() for word in words if word not in punct ]
    grams = [words[i:i+N] for i in range(len(words)-N+1)]

    for gram in grams: 
        count_tokens = []
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if joined_gram in unique_leagues:
            count_tokens.append(joined_gram)
    
    c = Counter(count_tokens)
    
    try:
        max_w1 = sorted(c, key=c.get)[-1]
        #print(max_w1)
        if df['league_name_in_russian'][i].lower() == max_w1:
            leagues_match[i] = 1
            #print(df['league_name_in_russian'][i].lower(), max_w1)
        
    except:
        continue

In [35]:
df['list_direct_match_for_leagues'] = leagues_match

In [36]:
success = df[df['list_direct_match_for_leagues'] == 1].shape[0]

In [37]:
all_vals = df['list_direct_match_for_leagues'].shape[0]

In [38]:
result_leagues = success / all_vals
print(result_leagues)

0.0


### русский

In [22]:
teams1_match = np.zeros(russian.shape[0])
teams2_match = np.zeros(russian.shape[0])

In [23]:
for i, text in tqdm(enumerate(russian['result.tsv'].values)):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            #print(token)
            if token in unique_teams:
                count_tokens.append(token)
        #print(count_tokens)
        c = Counter(count_tokens)
        #if count_tokens == 2
        #print(df['team_of_season_1_name_in_russian'][i], df['team_of_season_2_name_in_russian'][i])
        #print(c)
        #print(max(c))
        #print(c.most_common()[0:2])
        #print(text)
        try:
            max_w1 = sorted(c, key=c.get)[-1]
            #print(max_w1)
        except:
            continue
        try:
            max_w2 = sorted(c, key=c.get)[-2]
            #print(max_w2)
        except:
            continue
        
        #print('мэтч команды 1')
        if russian['team_of_season_1_name_in_russian'][i] == max_w1 or russian['team_of_season_1_name_in_russian'][i] == max_w2:
            teams1_match[i] = 1
            #print(df['team_of_season_1_name_in_russian'][i], max_w1)
            #print(df['team_of_season_1_name_in_russian'][i], max_w2)
        
        #print('мэтч команды 2')
        if russian['team_of_season_2_name_in_russian'][i] == max_w1 or russian['team_of_season_2_name_in_russian'][i] == max_w2:
            teams2_match[i] = 1
            #print(df['team_of_season_2_name_in_russian'][i], max_w1)
            #print(df['team_of_season_2_name_in_russian'][i], max_w2)
        

18it [00:03,  5.91it/s]


KeyError: 18

In [ ]:
russian['list_direct_match_for_teams1'] = teams1_match
russian['list_direct_match_for_teams2'] = teams2_match

In [ ]:
success1 = russian[russian['list_direct_match_for_teams1'] == 1].shape[0]
success2 = russian[russian['list_direct_match_for_teams2'] == 1].shape[0]

all_vals = russian['list_direct_match_for_teams1'].shape[0]

result_team1 = success1 / all_vals
print(result_team1)

result_team2 = success2 / all_vals
print(result_team2)

In [ ]:
russian.to_csv(r'C:\Users\User\Desktop\project\russian_lists_and_directmatch.csv', index = False)

In [19]:
unique_leagues

['Международные товарищеские матчи',
 'Кубок Америки',
 'Лига Европы УЕФА',
 'Международный Кубок чемпионов',
 'Премьер-Лига',
 'Клубные товарищеские матчи',
 'Чемпионат Европы. Квалификация',
 'Эредивизи',
 'Чемпионшип',
 'Премьер-лига',
 'Лига чемпионов УЕФА',
 'Примейра-лига',
 'КХЛ',
 'Премьершип',
 'Примера',
 'Лига Жюпилер',
 'Лига 1',
 'Серия А',
 'Бундеслига',
 'Суперкубок',
 'Кубок',
 'Кубок Испании',
 'Женский чемпионат',
 'Кубок Франции',
 'Кубок Либертадорес',
 'Кубок ФА',
 'Кубок АФЛ',
 'ФНЛ',
 'НХЛ',
 'Суперкубок УЕФА',
 'J-Лига',
 'А-Лига',
 'Серия B',
 'Суперлига',
 'Аллсвенскан',
 'Чемпионат мира U20',
 'Кубок африканских наций',
 'Чемпионат мира. Квалификация. Азия',
 'Вторая Бундеслига',
 'МЛС',
 'Элитсерия',
 'Чемпионат Европы U21',
 'Межконтинентальный кубок',
 'Чемпионат Европы U19 ',
 'Лига Наций УЕФА',
 'Сегунда',
 '1-й дивизион',
 'Экстракласа',
 'Первая Лига',
 'Лига чемпионов АФК',
 'Товарищеские матчи U21',
 'Кубок японской лиги',
 'Урвалсдейлд',
 'Кубок Йох

In [14]:
leagues_match = np.zeros(russian.shape[0])

In [21]:
%%time 
for i, text in tqdm(enumerate(russian['result.tsv'][:2].values)):
    n_words_league = len(word_tokenize(russian['league_name_in_russian'][i]))
    N = n_words_league
    words = word_tokenize(text)
    words = [word.lower() for word in words if word not in punct ]
    grams = [words[i:i+N] for i in range(len(words)-N+1)]
    #print(grams)

    for gram in grams: 
        count_tokens = []
        str1 = " "
        joined_gram = str1.join(gram)
        print(joined_gram)
        if joined_gram in unique_leagues:
            count_tokens.append(joined_gram)
    
    c = Counter(count_tokens)
    
    try:
        max_w1 = sorted(c, key=c.get)[-1]
        #print(max_w1)
        if russian['league_name_in_russian'][i].lower() == max_w1:
            leagues_match[i] = 1
            #print(df['league_name_in_russian'][i].lower(), max_w1)
        
    except:
        continue

0it [00:00, ?it/s]

сборная турции продолжает
турции продолжает свою
продолжает свою дорогу
свою дорогу на
дорогу на чемпионат
на чемпионат европы
чемпионат европы и
европы и перед
и перед очередными
перед очередными матчами
очередными матчами отбора
матчами отбора решила
отбора решила размяться
решила размяться в
размяться в товарищеском
в товарищеском матче
товарищеском матче узбекистан
матче узбекистан не
узбекистан не преследует
не преследует больших
преследует больших турнирных
больших турнирных задач
турнирных задач и
задач и просто
и просто сыграет
просто сыграет два
сыграет два товарищеских
два товарищеских матча
товарищеских матча за
матча за неделю
за неделю турки
неделю турки стали
турки стали самой
стали самой слабой
самой слабой командой
слабой командой группы
командой группы в2
группы в2 лиги
в2 лиги наций
лиги наций проиграв
наций проиграв три
проиграв три игры
три игры из
игры из четырех
из четырех тогда
четырех тогда команда
тогда команда финишировала
команда финишировала с
финишировала с

1it [00:01,  1.82s/it]

двадцать девятого
девятого числа
числа состоится
состоится один
один весьма
весьма престижный
престижный матч
матч в
в рамках
рамках четвертьфинальной
четвертьфинальной стадии
стадии кубка
кубка америки
америки столь
столь глубоко
глубоко позволила
позволила себе
себе колумбия
колумбия которой
которой будет
будет противостоять
противостоять чили
чили мероприятие
мероприятие начнется
начнется в
в два
два часа
часа ночи
ночи по
по московскому
московскому времени
времени на
на стадионе
стадионе коринтианс
коринтианс что
что расположен
расположен в
в сан-паулу
сан-паулу плавными
плавными шагами
шагами можно
можно переходить
переходить к
к оценке
оценке букмекерской
букмекерской линии
линии достойное
достойное событие
событие двух
двух достойных
достойных коллективов
коллективов даже
даже как-то
как-то обидно
обидно что
что придется
придется попрощаться
попрощаться с
с одной
одной из
из крепких
крепких команд
команд на
на четвертьфинальной
четвертьфинальной стадии
стадии говорить
говорить в

2it [00:03,  1.73s/it]


Wall time: 3.47 s
